In [57]:
s1="vV20MXMDsAE3CziS9d2N5S/80nDHRjStgsI9rzRFreL45qw0NIhKyiN/Mj2IBfm0KqGgSFaEuimoggy050xzAQ=="

In [164]:
import requests
import pandas as pd
import xml.etree.ElementTree as ET

# API 엔드포인트 및 파라미터 설정
url = "http://apis.data.go.kr/1741000/admmPpltnHhStus/selectAdmmPpltnHhStus"
params = {
    'serviceKey': s1,  # 실제 서비스 키로 교체
    'admmCd': '1114064500',
    'srchFrYm': '202404',
    'srchToYm': '202404',
    'lv': '4',
    'regSeCd': '1',
    'type': 'xml',
    'numOfRows': '100',
    'pageNo': '1'
}

# API로부터 데이터 가져오기
response = requests.get(url, params=params)

# 응답 내용 확인 및 예외 처리
if response.status_code == 200:
    if response.content:
        
        root = ET.fromstring(response.content)
        data = []

        # XML에서 필요한 정보 추출 (예: item 태그 내의 데이터)
        for item in root.findall('.//item'):
            record = {}
            for child in item:
                record[child.tag] = child.text
            data.append(record)

        # 데이터 프레임 생성
        df = pd.DataFrame(data)
df.columns=["달","행정코드","시도명", "시군구명", "행정동명", "통", "반", "총인구수", "세대수", "세대당인구", "남자인구수", "여자인구수", "남녀비율"]


In [165]:
df['통'] = df['통'].astype(int)
df['반'] = df['반'].astype(int)
df['총인구수'] = df['총인구수'].astype(int)
df['세대수'] = df['세대수'].astype(int)
df['세대당인구'] = df['세대당인구'].astype(float)
df['남자인구수'] = df['남자인구수'].astype(int)
df['여자인구수'] = df['여자인구수'].astype(int)
df['남녀비율'] = df['남녀비율'].astype(float)


In [166]:
df=pd.concat([df2,df])
df

,달,행정코드,시도명,시군구명,행정동명,통,반,총인구수,세대수,세대당인구,남자인구수,여자인구수,남녀비율
0,202404,1114064500,서울특별시,중구,청구동,14,6,91,41,2.22,36,55,0.65
1,202404,1114064500,서울특별시,중구,청구동,14,7,91,42,2.17,45,46,0.98
2,202404,1114064500,서울특별시,중구,청구동,14,8,86,40,2.15,39,47,0.83
3,202404,1114064500,서울특별시,중구,청구동,14,9,109,41,2.66,59,50,1.18
4,202404,1114064500,서울특별시,중구,청구동,14,10,94,47,2.00,44,50,0.88
...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,202404,1114064500,서울특별시,중구,청구동,14,1,99,41,2.41,46,53,0.87
96,202404,1114064500,서울특별시,중구,청구동,14,2,85,47,1.81,37,48,0.77
97,202404,1114064500,서울특별시,중구,청구동,14,3,93,48,1.94,41,52,0.79
98,202404,1114064500,서울특별시,중구,청구동,14,4,112,57,1.96,55,57,0.96


In [167]:
aggregated_df2 = df.groupby(['행정동명', '통'], as_index=False).agg({
    '총인구수': 'sum',
    '세대수': 'sum',
    '세대당인구': 'mean'
})

df_청구동=aggregated_df2[["행정동명","통","총인구수","세대수","세대당인구"]]
display(df_청구동)

,행정동명,통,총인구수,세대수,세대당인구
0,청구동,1,266,159,1.708333
1,청구동,2,292,165,1.780000
2,청구동,3,216,117,1.843333
3,청구동,4,399,220,1.797143
4,청구동,5,568,274,1.875000
5,청구동,6,344,154,2.230000
6,청구동,7,694,377,1.820000
7,청구동,8,554,344,1.616667
8,청구동,9,656,393,1.761667
9,청구동,10,268,199,1.378000


In [282]:
df_민태현=pd.concat([df_청구동,df_약수동,df_황학동])

In [170]:
df_민태현.to_csv("민태현.csv")

In [333]:
import pandas as pd
df=pd.read_csv("../data/통별인구수.csv")
df=df[["행정동명","통","총인구수","세대수"]]
df

,행정동명,통,총인구수,세대수
0,동화동,1,448,190
1,동화동,2,286,152
2,동화동,3,412,218
3,동화동,4,471,205
4,동화동,5,366,183
...,...,...,...,...
95,약수동,13,520,227
96,약수동,14,658,305
97,약수동,15,913,361
98,약수동,16,526,196


In [334]:
df=df.merge(df_민태현[["행정동명","통","총인구수","세대수"]],how="outer")

In [335]:
df['통'] = df['통'].astype(int)
df['총인구수'] = df['총인구수'].astype(int)
df['세대수'] = df['세대수'].astype(int)


In [338]:
df.query("행정동명=='신당동'")

,행정동명,통,총인구수,세대수
33,신당동,1,332,195
34,신당동,2,569,449
35,신당동,3,360,284
36,신당동,4,265,193
37,신당동,5,340,240
38,신당동,6,417,257
39,신당동,7,154,116
40,신당동,8,393,244
41,신당동,9,214,152
42,신당동,10,133,79


In [199]:
df_민태현=df_민태현.merge(df)

In [339]:
df_동통=pd.read_excel("C:/Users/vjame/OneDrive/OneDrive - dongguk.edu/문서/카카오톡 받은 파일/dongtong.xlsx")
df_동통.columns=["행정동명","통","노드"]

In [340]:
def add_dong_suffix(name):
    if not name.endswith('동'):
        return name + '동'
    return name

# '행정동명' 칼럼에 함수 적용
df_동통['행정동명'] = df_동통['행정동명'].apply(add_dong_suffix)
df_동통

,행정동명,통,노드
0,청구동,12,0
1,동화동,1,1
2,동화동,6,2
3,동화동,8,3
4,신당동,11,4
...,...,...,...
167,청구동,4,167
168,청구동,4,168
169,신당동,13,169
170,청구동,6,170


In [343]:
df_동통2=df_동통.groupby(["행정동명","통"]).count()


In [344]:
df_동통찐막=df_동통2.sort_values(["행정동명","통"]).reset_index()
df_동통찐막

,행정동명,통,노드
0,동화동,1,2
1,동화동,2,4
2,동화동,3,1
3,동화동,4,2
4,동화동,6,1
...,...,...,...
59,황학동,17,3
60,황학동,18,1
61,황학동,19,2
62,황학동,20,1


In [382]:
df['행정동명'] = df['행정동명'].apply(add_dong_suffix)
df["행정동명"].unique()

array(['동화동', '신당5동', '신당동', '황학동', '청구동', '약수동'], dtype=object)

In [383]:
df_동통찐막2=df_동통찐막.merge(df)
df_동통찐막2.head()

,행정동명,통,노드,총인구수,세대수
0,동화동,1,2,448,190
1,동화동,2,4,286,152
2,동화동,3,1,412,218
3,동화동,4,2,471,205
4,동화동,6,1,673,244


In [366]:
def add_dong_suffix(name):
    if name==('신당제5동'):
        return '신당5동'
    return name
df_동통['행정동명'] = df_동통['행정동명'].apply(add_dong_suffix)

In [367]:
df_동통찐막2["행정동명"].unique()

array(['동화동', '신당동', '약수동', '청구동', '황학동'], dtype=object)

In [384]:
sum(df_동통찐막2["총인구수"])
df_동통찐막2["비율"]=df_동통찐막2["총인구수"]/sum(df_동통찐막2["총인구수"])
df_동통찐막2["쓰레기양"]=41000*df_동통찐막2["비율"]/df_동통찐막2["노드"]
df_동통찐막2.head()

,행정동명,통,노드,총인구수,세대수,비율,쓰레기양
0,동화동,1,2,448,190,0.014967,306.828812
1,동화동,2,4,286,152,0.009555,97.938661
2,동화동,3,1,412,218,0.013765,564.345851
3,동화동,4,2,471,205,0.015736,322.581184
4,동화동,6,1,673,244,0.022484,921.856207


In [412]:
df_동통찐막2.query("통==17")

,행정동명,통,노드,총인구수,세대수,비율,쓰레기양
13,신당5동,17,3,203,147,0.006782,92.687870
34,약수동,17,1,107,39,0.003575,146.565549
35,약수동,17,1,778,276,0.025992,1065.682213
46,청구동,17,1,1070,371,0.035748,1465.655486
61,황학동,17,3,247,150,0.008252,112.777852


In [413]:
df_동통찐막2 = df_동통찐막2.drop(df_동통찐막2.index[34])

In [385]:
df_동통["행정동명"].unique()

array(['청구동', '동화동', '신당동', '황학동', '약수동', '신당5동'], dtype=object)

In [386]:
df_동통찐막2["행정동명"].unique()

array(['동화동', '신당5동', '신당동', '약수동', '청구동', '황학동'], dtype=object)

In [414]:
np.where(df_동통.merge(df_동통찐막2,how="outer",on=["행정동명","통"]).sort_values("노드_x").groupby("노드_x").count()["행정동명"]>=2)

(array([], dtype=int64),)

In [420]:
df=df_동통.merge(df_동통찐막2,how="outer",on=["행정동명","통"]).sort_values("노드_x")

In [426]:
df[["노드_x","행정동명","통","총인구수","쓰레기양"]].to_csv("노드별_인구수_쓰레기양.csv")

In [432]:
lst=df["쓰레기양"].to_list()
demands_int = list(map(int, lst))


In [433]:
demands_int

[612,
 306,
 921,
 982,
 231,
 1068,
 173,
 151,
 35,
 612,
 122,
 122,
 140,
 414,
 45,
 182,
 224,
 122,
 306,
 45,
 45,
 136,
 224,
 133,
 60,
 140,
 61,
 538,
 136,
 758,
 224,
 196,
 61,
 45,
 820,
 97,
 35,
 61,
 285,
 97,
 224,
 232,
 232,
 98,
 98,
 52,
 605,
 97,
 133,
 140,
 140,
 140,
 97,
 136,
 97,
 136,
 136,
 97,
 133,
 98,
 61,
 215,
 125,
 136,
 136,
 94,
 94,
 136,
 280,
 94,
 97,
 173,
 1465,
 35,
 285,
 384,
 217,
 332,
 258,
 984,
 112,
 73,
 73,
 151,
 332,
 1043,
 440,
 440,
 196,
 258,
 760,
 136,
 332,
 112,
 378,
 196,
 52,
 112,
 1010,
 151,
 71,
 950,
 160,
 378,
 432,
 280,
 605,
 210,
 35,
 35,
 35,
 125,
 217,
 173,
 71,
 322,
 322,
 436,
 479,
 1065,
 387,
 173,
 322,
 322,
 71,
 564,
 423,
 432,
 215,
 215,
 92,
 52,
 160,
 52,
 92,
 92,
 160,
 160,
 280,
 397,
 35,
 35,
 35,
 35,
 35,
 35,
 35,
 35,
 35,
 1231,
 61,
 61,
 60,
 35,
 35,
 35,
 35,
 35,
 35,
 60,
 61,
 94,
 35,
 60,
 61,
 182,
 60,
 136,
 136,
 414,
 94,
 60]

In [332]:
df_동통.merge(df,how="left",left_on=["행정동명","통"],right_on=["행정동명","통"]).sort_values("노드_x").iloc[60:90]

,행정동명,통,노드_x,총인구수,세대수,노드_y,비율,쓰레기양
60,신당동,3,60,360.0,284.0,8.0,0.013298,68.151596
61,신당5동,15,61,NaN,NaN,NaN,NaN,NaN
62,황학동,13,62,183.0,157.0,2.0,0.006760,138.574911
63,신당동,19,63,699.0,622.0,7.0,0.025820,151.231636
64,신당동,19,64,699.0,622.0,7.0,0.025820,151.231636
65,청구동,6,65,344.0,154.0,5.0,0.012707,104.196217
66,청구동,6,66,344.0,154.0,5.0,0.012707,104.196217
67,신당동,19,67,699.0,622.0,7.0,0.025820,151.231636
68,신당5동,5,68,NaN,NaN,NaN,NaN,NaN
69,청구동,6,69,344.0,154.0,5.0,0.012707,104.196217


In [326]:
df_동통.head(30)

,행정동명,통,노드
0,청구동,12,0
1,동화동,1,1
2,동화동,6,2
3,동화동,8,3
4,신당동,11,4
5,황학동,26,5
6,약수동,7,6
7,신당동,1,7
8,신당동,2,8
9,청구동,12,9


In [315]:
df.replace(np.inf, np.nan, inplace=True)
df.dropna(inplace=True)


In [316]:
df

,행정동명,통,총인구수,세대수,노드,비율,쓰레기양
0,동화동,1,448.0,190.0,2.0,0.006771,138.800308
1,동화동,2,286.0,152.0,4.0,0.004322,44.304563
2,동화동,3,412.0,218.0,1.0,0.006227,255.293424
3,동화동,4,471.0,205.0,2.0,0.007118,145.926217
5,동화동,6,673.0,244.0,1.0,0.010171,417.020569
...,...,...,...,...,...,...,...
128,황학동,17,247.0,150.0,3.0,0.003733,51.017375
129,황학동,18,719.0,473.0,1.0,0.010866,445.524204
130,황학동,19,318.0,231.0,2.0,0.004806,98.523433
131,황학동,20,738.0,319.0,1.0,0.011154,457.297444


In [ ]:
pd.DataFrame("거점번호":,"도로명주소","도로명인구수","몇m이내 몇개","쓰레기양")
                                           10           2        
1번거점에 인구수가 10명이고 도로명안에 같은 거점에 2개 있음   ->    쓰레기양 =    
전체 쓰레기량 / 전체 인구수 * 10 / 2  

전체  쓰레기양 (톤) / 중구 인구수 

------------------------------------------
L -> 무게
1L -> 무게